# Tensorflow Load and Benchmark Tests

Using a pretrained model for [Tensorflow flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers)

 * Load test the model at fixed rate
 * Benchmark the model to find maximum throughput and saturation handling
 
 ## Setup
 
  * Create a 1 node GCP cluster with n1-standard-8 node
  * Install Seldon Core

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_seldon-demos_europe-west4-a_cluster-2" modified.


In [1]:
import sys
sys.path.append('../')
from vegeta_utils import *

## Benchmark with Saturation Test

In [12]:
%%writefile tf_flowers.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tf-flowers
spec:
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      implementation: TENSORFLOW_SERVER
      modelUri: gs://kfserving-samples/models/tensorflow/flowers
      name:  flowers
      parameters:
        - name: model_name
          type: STRING
          value: flowers
    componentSpecs:
    - spec:
        containers:
        - name: flowers
          resources:
            requests:
              cpu: '2'
    name: default
    replicas: 1

Overwriting tf_flowers.yaml


In [13]:
run_model("tf_flowers.yaml")

Failed
Available


Run test to gather the max throughput of the model

In [10]:
results = run_test("tf_vegeta_cfg.yaml","vegeta_max.yaml","11m")
print(json.dumps(results, indent=4))
saturation_throughput=int(results["throughput"])
print("Max Throughtput=",saturation_throughput)

KeyboardInterrupt: 

In [ ]:
print("Max Throughtput=",saturation_throughput)

## Load Tests with HPA

Run with an HPA at saturation rate to check:
  * Latencies affected by scaling


In [5]:
%%writefile tf_flowers.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tf-flowers
spec:
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      implementation: TENSORFLOW_SERVER
      modelUri: gs://kfserving-samples/models/tensorflow/flowers
      name:  flowers
      parameters:
        - name: model_name
          type: STRING
          value: flowers
    componentSpecs:
    - hpaSpec:
        minReplicas: 1
        maxReplicas: 5
        metrics:
        - resource:
            name: cpu
            targetAverageUtilization: 70
          type: Resource
      spec:
        containers:
        - name: flowers
          resources:
            requests:
              cpu: '1'
          livenessProbe:
            failureThreshold: 3
            initialDelaySeconds: 60
            periodSeconds: 5
            successThreshold: 1
            tcpSocket:
              port: http
            timeoutSeconds: 5
          readinessProbe:
            failureThreshold: 3
            initialDelaySeconds: 20
            periodSeconds: 5
            successThreshold: 1
            tcpSocket:
              port: http
            timeoutSeconds: 5
    name: default
    replicas: 1

Overwriting tf_flowers.yaml


In [6]:
run_model("tf_flowers.yaml")

Available


In [7]:
rate=saturation_throughput
duration="10m"
%env DURATION=$duration
%env RATE=$rate/1s
!cat vegeta_cfg.tmpl.yaml | envsubst > vegeta.tmp.yaml
!cat vegeta.tmp.yaml

env: DURATION=10m
env: RATE=18/1s
apiVersion: batch/v1
kind: Job
metadata:
  name: tf-load-test
spec:
  backoffLimit: 6
  parallelism: 1
  template:
    metadata:
      annotations:
        sidecar.istio.io/inject: "false"
    spec:
      containers:
        - args:
            - vegeta -cpus=4 attack -keepalive=false -duration=10m -rate=18/1s -targets=/var/vegeta/cfg
              | vegeta report -type=json
          command:
            - sh
            - -c
          image: peterevans/vegeta:latest
          imagePullPolicy: Always
          name: vegeta
          volumeMounts:
            - mountPath: /var/vegeta
              name: tf-vegeta-cfg
      restartPolicy: Never
      volumes:
        - configMap:
            defaultMode: 420
            name: tf-vegeta-cfg
          name: tf-vegeta-cfg


In [8]:
results = run_test("tf_vegeta_cfg.yaml","vegeta.tmp.yaml","11m")
print(json.dumps(results, indent=4))

{
    "latencies": {
        "total": 6235317971355,
        "mean": 577344256,
        "50th": 114263582,
        "90th": 1927105619,
        "95th": 4408763192,
        "99th": 6560764472,
        "max": 11078287773,
        "min": 80305724
    },
    "bytes_in": {
        "total": 2354400,
        "mean": 218
    },
    "bytes_out": {
        "total": 174690000,
        "mean": 16175
    },
    "earliest": "2020-07-09T07:01:17.395810851Z",
    "latest": "2020-07-09T07:11:17.340159956Z",
    "end": "2020-07-09T07:11:17.475663766Z",
    "duration": 599944349105,
    "wait": 135503810,
    "requests": 10800,
    "rate": 18.00166968171547,
    "throughput": 17.99760473133198,
    "success": 1,
    "status_codes": {
        "200": 10800
    },
    "errors": []
}


In [9]:
print_results(results)

Latencies:
	mean: 577.344256 ms
	50th: 114.263582 ms
	90th: 1927.105619 ms
	95th: 4408.763192 ms
	99th: 6560.764472 ms

Throughput: 17.99760473133198/s
Errors: False
